### Import Required Libraries and Set Up Environment Variables

In [52]:
# Dependencies
import requests
import json
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv
from prettytable import PrettyTable

## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [53]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
specifier = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate = "2024-05-01"
api_key = "xV9MOc6zaRwpUFMt1TrYB62R6GtI5eh5WxJkJYbt"

# Build URL for CME
query_url = f"{base_url}{specifier}?startDate={startDate}&endDate={endDate}&api_key={api_key}"


In [54]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url)

In [55]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [56]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print("Preview of JSON data:")
print(json.dumps(cme_json[:2], indent=4))

Preview of JSON data:
[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "s

In [57]:
# Convert cme_json to a Pandas DataFrame 
if isinstance(cme_json, list) and len(cme_json) > 0:
    cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
    required_columns = ['activityID', 'startTime', 'linkedEvents']
    missing_columns = [col for col in required_columns if col not in cme_df.columns]

    if missing_columns:
        raise KeyError(f"One or more required columns are missing from the data: {missing_columns}")

    cme_df = cme_df[required_columns]
else:
    raise ValueError("The response JSON is not a list or is empty")

print("\nInitial DataFrame after keeping only required columns:")
print(cme_df.head())



Initial DataFrame after keeping only required columns:
                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-02T05:24:00-CME-001  2013-05-02T05:24Z   
2  2013-05-02T14:36:00-CME-001  2013-05-02T14:36Z   
3  2013-05-03T18:00:00-CME-001  2013-05-03T18:00Z   
4  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   

                                      linkedEvents  
0  [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
1                                             None  
2                                             None  
3                                             None  
4  [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  


In [58]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])

print("\nDataFrame after dropping rows with missing 'linkedEvents':")
print(cme_df.head())


DataFrame after dropping rows with missing 'linkedEvents':
                     activityID          startTime  \
0   2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
4   2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
7   2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
10  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
13  2013-05-13T16:18:00-CME-001  2013-05-13T16:18Z   

                                         linkedEvents  
0     [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
4     [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  
7     [{'activityID': '2013-05-12T23:30:00-IPS-001'}]  
10  [{'activityID': '2013-05-13T01:53:00-FLR-001'}...  
13  [{'activityID': '2013-05-13T15:40:00-FLR-001'}...  


In [59]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    activity_id = cme_df.at[i, 'activityID']
    start_time = cme_df.at[i, 'startTime']
    linked_events = cme_df.at[i, 'linkedEvents']
    # Iterate over each dictionary in the list
    for item in linked_events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      expanded_rows.append({
         'activityID': activity_id,
         'startTime': start_time,
         'linkedEvent': item
      })
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

print("\nExpanded DataFrame:")
print(expanded_df.head())


Expanded DataFrame:
                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  


In [60]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        # Extract the 'activity_id' key
        return input_dict.get('activityID')
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error extracting activityID: {e}")
        return None
    
# Verification that it works
if not cme_df.empty:
    example_linked_event = cme_df.at[0, 'linkedEvents'][0]
    print("\nExtracted activityID from example linkedEvent:")
    print(extract_activityID_from_dict(example_linked_event))




Extracted activityID from example linkedEvent:
2013-05-04T04:52:00-IPS-001


In [61]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_df['GST_ActivityID'] = expanded_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict(x))

print("\nDataFrame after applying extract_activityID_from_dict to linkedEvent:")
print(expanded_df.head())


DataFrame after applying extract_activityID_from_dict to linkedEvent:
                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent               GST_ActivityID  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  2013-05-04T04:52:00-IPS-001  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  2013-05-07T04:37:00-IPS-001  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  2013-05-12T23:30:00-IPS-001  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  2013-05-13T01:53:00-FLR-001  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  2013-05-13T04:12:00-SEP-001  


In [62]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_df = expanded_df.dropna(subset=['GST_ActivityID'])

print("\nDataFrame after dropping rows with missing 'GST_ActivityID':")
print(expanded_df.head())


DataFrame after dropping rows with missing 'GST_ActivityID':
                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent               GST_ActivityID  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  2013-05-04T04:52:00-IPS-001  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  2013-05-07T04:37:00-IPS-001  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  2013-05-12T23:30:00-IPS-001  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  2013-05-13T01:53:00-FLR-001  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  2013-05-13T04:12:00-SEP-001  


In [63]:
# print out the datatype of each column in this DataFrame:
print(expanded_df.dtypes)

activityID        object
startTime         object
linkedEvent       object
GST_ActivityID    object
dtype: object


In [64]:
# Convert the 'GST_ActivityID' column to string format 
expanded_df['GST_ActivityID'] = expanded_df['GST_ActivityID'].astype(str)
# Convert startTime to datetime format  
expanded_df['startTime'] = pd.to_datetime(expanded_df['startTime'])
# Rename startTime to startTime_CME and activityID to cmeID
expanded_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)
# Drop linkedEvents
expanded_df = expanded_df.drop(columns=['linkedEvent'])
# Verify that all steps were executed correctly
print(expanded_df.head())
print(expanded_df.dtypes)
print("\nDataFrame after converting types, renaming columns, and dropping 'linkedEvents':")
print(expanded_df.head())

                         cmeID             startTime_CME  \
0  2013-05-01T03:12:00-CME-001 2013-05-01 03:12:00+00:00   
1  2013-05-03T22:36:00-CME-001 2013-05-03 22:36:00+00:00   
2  2013-05-09T19:29:00-CME-001 2013-05-09 19:29:00+00:00   
3  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   
4  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   

                GST_ActivityID  
0  2013-05-04T04:52:00-IPS-001  
1  2013-05-07T04:37:00-IPS-001  
2  2013-05-12T23:30:00-IPS-001  
3  2013-05-13T01:53:00-FLR-001  
4  2013-05-13T04:12:00-SEP-001  
cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object

DataFrame after converting types, renaming columns, and dropping 'linkedEvents':
                         cmeID             startTime_CME  \
0  2013-05-01T03:12:00-CME-001 2013-05-01 03:12:00+00:00   
1  2013-05-03T22:36:00-CME-001 2013-05-03 22:36:00+00:00   
2  2013-05-09T19:29:00-CME-001 2013-05-09 19:29:00+

In [65]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
expanded_df = expanded_df[expanded_df['GST_ActivityID'].str.contains('GST')]

print("\nFiltered DataFrame containing only rows where 'GST_ActivityID' contains 'GST':")
print(expanded_df.head())



Filtered DataFrame containing only rows where 'GST_ActivityID' contains 'GST':
                           cmeID             startTime_CME  \
21   2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
48   2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00   
90   2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00   
148  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00   
151  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00   

                  GST_ActivityID  
21   2013-06-07T03:00:00-GST-001  
48   2013-10-02T03:00:00-GST-001  
90   2013-12-08T00:00:00-GST-001  
148  2014-02-19T03:00:00-GST-001  
151  2014-02-20T03:00:00-GST-001  


In [66]:
# Pretty Table utilization for better view of data. 
print("\nFiltered DataFrame containing only rows where 'GST_ActivityID' contains 'GST':")
pretty_table = PrettyTable()
pretty_table.field_names = list(expanded_df.columns)

for row in expanded_df.head().itertuples(index=False):
    pretty_table.add_row(list(row))

print(pretty_table)


Filtered DataFrame containing only rows where 'GST_ActivityID' contains 'GST':
+-----------------------------+---------------------------+-----------------------------+
|            cmeID            |       startTime_CME       |        GST_ActivityID       |
+-----------------------------+---------------------------+-----------------------------+
| 2013-06-02T20:24:00-CME-001 | 2013-06-02 20:24:00+00:00 | 2013-06-07T03:00:00-GST-001 |
| 2013-09-29T22:40:00-CME-001 | 2013-09-29 22:40:00+00:00 | 2013-10-02T03:00:00-GST-001 |
| 2013-12-04T23:12:00-CME-001 | 2013-12-04 23:12:00+00:00 | 2013-12-08T00:00:00-GST-001 |
| 2014-02-16T14:15:00-CME-001 | 2014-02-16 14:15:00+00:00 | 2014-02-19T03:00:00-GST-001 |
| 2014-02-18T01:25:00-CME-001 | 2014-02-18 01:25:00+00:00 | 2014-02-20T03:00:00-GST-001 |
+-----------------------------+---------------------------+-----------------------------+


### GST Data

In [67]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_query_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={api_key}"
print("GST Query URL:", gst_query_url)


GST Query URL: https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=xV9MOc6zaRwpUFMt1TrYB62R6GtI5eh5WxJkJYbt


In [68]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
get_response = requests.get(gst_query_url)
print("Status Code", get_response.status_code)
print("\nGST Response Object:", get_response)

Status Code 200

GST Response Object: <Response [200]>


In [76]:
import requests
import json
import os
from dotenv import load_dotenv

# Load the NASA_API_KEY from the .env file
load_dotenv()
api_key = os.getenv('NASA_API_KEY')

# Check if API key is loaded correctly
if not api_key:
    raise ValueError("NASA_API_KEY not found. Please check your .env file.")

# Step 1: Set the base URL and parameters for GST
base_url = "https://api.nasa.gov/DONKI/"
GST = "GST"
startDate = "2013-05-01"
endDate = "2024-05-01"

# Step 2: Build the query URL for GST
gst_query_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={api_key}"

# Step 3: Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_query_url)

# Print the status code to verify if the request was successful
print("Status Code:", gst_response.status_code)

# Step 4: Convert the response variable to JSON and store it as a variable named gst_json
if gst_response.status_code == 200:
    gst_json = gst_response.json()

    # Preview the first result in JSON format
    print("\nPreview of GST JSON data:")
    print(json.dumps(gst_json[:2], indent=4))  # Print the first two items to preview the structure
else:
    print(f"Error: Request failed with status code {gst_response.status_code}")



Status Code: 200

Preview of GST JSON data:
[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
    

In [81]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)
# Keep only the columns: activityID, startTime, linkedEvents
required_columns = ['gstID', 'startTime', 'linkedEvents']
# Checking Columns 
missing_columns = [col for col in required_columns if col not in gst_df.columns]
if missing_columns:
    raise KeyError(f"One or more required columns are missing from the data: {missing_columns}")

# Required Columns
gst_df = gst_df[required_columns]

# Rows Displayed
print("\nGST DataFrame after keeping only required columns:")
print(gst_df.head())


GST DataFrame after keeping only required columns:
                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-06-29T03:00:00-GST-001  2013-06-29T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   

                                        linkedEvents  
0    [{'activityID': '2013-05-31T15:45:00-HSS-001'}]  
1    [{'activityID': '2013-06-02T20:24:00-CME-001'}]  
2                                               None  
3  [{'activityID': '2013-09-29T22:40:00-CME-001'}...  
4  [{'activityID': '2013-12-04T23:12:00-CME-001'}...  


In [82]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])

print("\nGST DataFrame after removing rows with missing 'linked_events':")
print(gst_df.head())


GST DataFrame after removing rows with missing 'linked_events':
                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   
5  2014-02-19T03:00:00-GST-001  2014-02-19T03:00Z   

                                        linkedEvents  
0    [{'activityID': '2013-05-31T15:45:00-HSS-001'}]  
1    [{'activityID': '2013-06-02T20:24:00-CME-001'}]  
3  [{'activityID': '2013-09-29T22:40:00-CME-001'}...  
4  [{'activityID': '2013-12-04T23:12:00-CME-001'}...  
5  [{'activityID': '2014-02-16T14:15:00-CME-001'}...  


In [83]:
# Notice that the linkedEvents sometimes contains multiple events per row
gst_df['linkedEvents'] = gst_df['linkedEvents'].apply(lambda x: x if isinstance(x, list) else [x])
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.dropna(subset=['linkedEvents'])

print("\nGST DataFrame after exploding 'linkedEvents' into individual rows:")
print(gst_df.head())


GST DataFrame after exploding 'linkedEvents' into individual rows:
                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  


In [87]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict.get('activityID')
    except (ValueError, TypeError) as e:
        print(f"Error extracting activityID: {e}")
        return None
# Apply the function to each row in 'linkedEvents' column and create a new column called 'CME_ActivityID
gst_df ['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
# Rows removed with the missing CME_activity ID
gst_df = gst_df.dropna(subset=['CME_ActivityID'])
# Display DataFrame to verify Changes 
print("\nGST DataFrame after applying 'extract_activityID_from_dict' and removing missing 'CME_ActivityID':")
print(gst_df.head())


GST DataFrame after applying 'extract_activityID_from_dict' and removing missing 'CME_ActivityID':
                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents               CME_ActivityID  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  2013-05-31T15:45:00-HSS-001  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  2013-06-02T20:24:00-CME-001  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  2013-09-29T22:40:00-CME-001  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  2013-10-02T01:54:00-IPS-001  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  2013-10-02T02:47:00-MPC-001  


In [94]:
# Convert 'CME_ActivityID' to string format
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert 'gstID' to string format
gst_df['gstID'] = gst_df['gstID'].astype(str)

# Convert startTime to datetime format if available
if 'startTime' in gst_df.columns:
    gst_df['startTime_GST'] = pd.to_datetime(gst_df['startTime'], errors='coerce')
    # Remove rows with invalid datetime after coercion
    gst_df = gst_df.dropna(subset=['startTime_GST'])

# Drop linkedEvents
if 'linkedEvents' in gst_df.columns:
    gst_df = gst_df.drop(columns=['linkedEvents'])

# Verify the resulting DataFrame
print("\nGST DataFrame after final data cleaning:")
print(gst_df.head())



GST DataFrame after final data cleaning:
                         gstID      startTime_GST  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                CME_ActivityID             startTime_GST  
0  2013-05-31T15:45:00-HSS-001 2013-06-01 01:00:00+00:00  
1  2013-06-02T20:24:00-CME-001 2013-06-07 03:00:00+00:00  
2  2013-09-29T22:40:00-CME-001 2013-10-02 03:00:00+00:00  
3  2013-10-02T01:54:00-IPS-001 2013-10-02 03:00:00+00:00  
4  2013-10-02T02:47:00-MPC-001 2013-10-02 03:00:00+00:00  


In [95]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
if 'CME_ActivityID' in gst_df.columns:
    gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')]

### Merge both datatsets

In [101]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_df, expanded_df, left_on=['gstID', 'CME_ActivityID'], right_on=['GST_ActivityID', 'cmeID'], how='inner')

# Drop duplicate merged DataFrame
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

print("\nMerged DataFrame containing GST and CME data:")
print(merged_df.head())


Merged DataFrame containing GST and CME data:
                         gstID      startTime_GST  \
0  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
1  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
2  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   
3  2014-02-19T03:00:00-GST-001  2014-02-19T03:00Z   
4  2014-02-20T03:00:00-GST-001  2014-02-20T03:00Z   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001  
2 2013-12-04 23:12:00+00:00  2013-12-08T00:00:00-GST-001  
3 2014-02-16 14:15:

In [102]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(f"Number of rows in CME DataFrame: {len(expanded_df)}")
print(f"Number of rows in GST DataFrame: {len(gst_df)}")
print(f"Number of rows in Merged DataFrame: {len(merged_df)}")

Number of rows in CME DataFrame: 61
Number of rows in GST DataFrame: 61
Number of rows in Merged DataFrame: 61


### Computing the time it takes for a CME to cause a GST

In [105]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`
merged_df['startTime_GST'] = pd.to_datetime(merged_df['startTime_GST'], errors='coerce')
merged_df['startTime_CME'] = pd.to_datetime(merged_df['startTime_CME'], errors='coerce')
merged_df['timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime_CME']).abs()

# Verify the updated DataFrame
print("\nMerged DataFrame with time difference:")
print(merged_df.head())


Merged DataFrame with time difference:
                         gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID        timeDiff  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001 4 days 06:36:00  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001 2 days 04:20:

In [107]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
print("\nStatistics for time difference between CME and GST:")
time_diff_stats = merged_df['timeDiff'].describe()
print(time_diff_stats)

mean_time_diff = time_diff_stats['mean']
median_time_diff = merged_df['timeDiff'].median()
print(f"\nMean time difference: {mean_time_diff}")
print(f"Median time difference: {median_time_diff}")


Statistics for time difference between CME and GST:
count                           61
mean     2 days 23:29:26.557377049
std      0 days 23:53:09.336914240
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 19:51:00
75%                3 days 13:35:00
max                6 days 03:00:00
Name: timeDiff, dtype: object

Mean time difference: 2 days 23:29:26.557377049
Median time difference: 2 days 19:51:00


### Exporting data in csv format

In [108]:
# Export data to CSV without the index
merged_df.to_csv('merged_gst_cme_data.csv', index=False)
print("\nMerged DataFrame exported to 'merged_gst_cme_data.csv'")



Merged DataFrame exported to 'merged_gst_cme_data.csv'
